In [1]:
#使用ResNet18初始化，使用自定义模型训练（多次多头注意力迭代）
#更改Wx输出后的大小
#

In [2]:
import matplotlib.pyplot as plt
from torch.cuda.amp import autocast 
from torch.cuda.amp import GradScaler
import torch
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch import optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import torch.nn as nn
import torchvision
from torchsummary import summary
import time
from torch.nn import init
from typing import Union, List, Dict, Any, Optional, cast
import warnings
warnings.filterwarnings("ignore")

In [3]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

In [4]:
#保存训练数据和模型
data_csv_path = "D:\\OneModel\\qyxx-resnet18.csv"   #修改此处文件名 
model_save_path = "D:\\OneModel\\qyxx-resnet18.pkl"  #修改此处文件名

In [5]:
train_path = "D:\\Dataset\\RAF-DB\\train"
val_path = "D:\\Dataset\\RAF-DB\\test"
#模型批次大小
batch_size = 128
resume = True
#动态学习率，学习率和循环次数增加
lr = 5e-4
epochs = 200
D_epoch = 0 
best_acc  = 0
print("epochs:",epochs,"learning_rate:",lr,"batch_size:",batch_size)

epochs: 200 learning_rate: 0.0005 batch_size: 128


In [6]:
#设备选取
flag = torch.cuda.is_available()
if flag:
    print("GPU")
else:
    print("CPU")
ngpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
# device = torch.device("cpu")
#查看显卡名称
#torch.cuda.get_device_name()
print("divice is ", device)

#数据预处理（建议提前resize，减少每次资源的损失）放大到112x112 ，随机水平翻转
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

#增加不同种transform，预测集中去除随机翻转
val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

GPU
divice is  cuda:0


In [7]:
from torchvision import models
resnet = models.resnet18(pretrained=True)

resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
class SKNet(nn.Module):
    def __init__(self, num_class=7):
        super(SKNet, self).__init__()
        self.features = nn.Sequential(*list(resnet.children())[:-2])
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(512, num_class)

    def forward(self, x):
        
        x = self.features(x)
        out = self.avgpool(x)
        out = torch.flatten(out,1)
        out = self.fc(out)
        return out

In [9]:
#使用torchvision.datasets.ImageFolder读取数据集指定train和test文件夹
train_data = torchvision.datasets.ImageFolder(train_path, transform=train_transform)
#drop_last舍弃未满一个批次的数据        num_workers工作区一般设置为GPU个数的4倍
data0_train = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True,num_workers=4)
print(train_data)  #输出训练集相关
val_data = torchvision.datasets.ImageFolder(val_path, transform=val_transform)
data1_val = DataLoader(val_data, batch_size=batch_size, shuffle=True,drop_last=True,num_workers=4)
print(val_data)  #输出测试集相关

print(train_data.classes)  #根据分的文件夹的名字来确定的类别
print(train_data.class_to_idx) #按顺序为这些类别定义索引为0,1...
print()
print(val_data.classes)
print(val_data.class_to_idx)
print()


Dataset ImageFolder
    Number of datapoints: 12271
    Root location: D:\Dataset\RAF-DB\train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
Dataset ImageFolder
    Number of datapoints: 3068
    Root location: D:\Dataset\RAF-DB\test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sadness', 'Surprise']
{'Anger': 0, 'Disgust': 1, 'Fear': 2, 'Happiness': 3, 'Neutral': 4, 'Sadness': 5, 'Surprise': 6}

['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sadness', 'Surprise']
{'Anger':

In [10]:
#to(device)将模型加入GPU中加速计算
model = SKNet().to(device)
#设置优化器

params = list(model.parameters()) 
optimizer = optim.AdamW(params, lr=lr)

# optimizer = optim.AdamW(model.parameters(), lr=lr)
#设置损失函数
criteon = nn.CrossEntropyLoss().to(device)
#余弦衰减学习率
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=80, eta_min=0)
#形如TensorFlow中的summary函数输出模型参数
summary(model, input_size=[(3, 224, 224)], batch_size=batch_size, device="cuda")
print()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [128, 64, 112, 112]           9,408
       BatchNorm2d-2        [128, 64, 112, 112]             128
              ReLU-3        [128, 64, 112, 112]               0
         MaxPool2d-4          [128, 64, 56, 56]               0
            Conv2d-5          [128, 64, 56, 56]          36,864
       BatchNorm2d-6          [128, 64, 56, 56]             128
              ReLU-7          [128, 64, 56, 56]               0
            Conv2d-8          [128, 64, 56, 56]          36,864
       BatchNorm2d-9          [128, 64, 56, 56]             128
             ReLU-10          [128, 64, 56, 56]               0
       BasicBlock-11          [128, 64, 56, 56]               0
           Conv2d-12          [128, 64, 56, 56]          36,864
      BatchNorm2d-13          [128, 64, 56, 56]             128
             ReLU-14          [128, 64,

In [11]:
#测试函数
def evalute_(model,val_loader):
    model.eval()
    test_loss2 = 0.0
    test_corrects2 = 0.0
    number = 0
    for batchidx, (x, label) in enumerate(val_loader):
#         print(number)
    #torch.cuda.empty_cache()  #清除非必要GPU缓存，但是我建议不要在训练中使用此句，这可能会损失你相当多的时间
        number = number + 1
        x, label = x.to(device), label.to(device)
        #测试函数中加入no_grad()，如果不加会增加计算和显存
        with torch.no_grad():
            y1 = model(x)
            #虽然可以直接使用max函数，但是我建议在y1的比较重你最好使用F.softmax(y1,dim=1)，这样可能会有更好的效果，我在训练中使用了它
            _, preds1 = torch.max(F.softmax(y1,dim=1), 1)
            loss = criteon(y1, label)  
            
            test_loss2 += loss.item()*batch_size
            test_corrects2 += torch.sum(preds1 == label.data)
    #由于使用了最后一次抛弃，我不能使用全部测试集作为分母，这样会使最后的准确率变小
    test_loss1 = test_loss2 / (number*batch_size)
    test_acc1 = test_corrects2.double() / (number*batch_size)
#     print("TestDataset loss is ", test_loss1,"TestDataset accuracy is ",test_acc1)
    return test_acc1, test_loss1
print("执行结束")

执行结束


In [12]:
#关于AMP自动精度求解，我也并不是很熟悉，只能使用官方给的实例进行照葫芦画瓢。
scaler = torch.cuda.amp.GradScaler()
torch.cuda.empty_cache()
for epoch in range(D_epoch, epochs):
    time_one = time.time()                         #标记训练开始时间戳
    train_acc1 = 0.0
    train_loss1 = 0.0
    train_acc = 0.0
    train_loss = 0.0
    
    val_acc = 0.0
    val_loss = 0.0
    number = 0
    model.train()
    print("epoch:",epoch)

    for batchidx , (x ,label) in enumerate(data0_train):
        x , label = x.to(device), label.to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            y1 = model(x)
            loss = criteon(y1,label) 
        _, preds1 = torch.max(F.softmax(y1,dim=1), 1)
        #AMP优化
        scaler.scale(loss).backward()
#         loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 5, norm_type=2)  #梯度剪裁
        scaler.step(optimizer)
#         optimizer.step()
        scaler.update()
        train_loss1 += loss.item()*batch_size
        train_acc1 += torch.sum(preds1 == label.data).double()
        number = number + 1
    time_two = time.time()             #标记训练结束时间戳
    #输出训练一轮所需时间，用于分析对比
    print("The Model-Train-Time spent  %d min %.2f s"%((time_two-time_one)//60,(time_two-time_one)%60))
    #计算训练时候的平均损失和平均准确率
    train_loss = train_loss1 / (number*batch_size)
    train_acc = train_acc1 / (number*batch_size)
    #计算测试时候的平均损失和平均准确率
    val_acc, val_loss = evalute_(model, data1_val)
    
    train_acc = train_acc.cpu()
    val_acc = val_acc.cpu()
    print('Accuracy : Train is {} , Valid is {} ;  Loss : Train is  {} ,Valid is {}'.format(train_acc, val_acc, train_loss , val_loss))
    #如果你不需要训练以及验证的准确率和损失值，你可以注释这下面的两行，它们不是非必须的，理论上只存在于汇报和论文中
    dataframe = pd.DataFrame(columns = [epoch,train_acc,train_loss,val_acc, val_loss])
    dataframe.to_csv(data_csv_path,line_terminator="\n",mode='a',index=False,sep=',')
    if val_acc > best_acc:
        print("覆盖最好的模型...")
        best_acc = val_acc 
        checkpoint = {
            'epoch': epoch,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'best_acc':best_acc
        }
        torch.save(checkpoint,model_save_path)
#     time_three = time.time() 
#     print("测试花费时间",time_three-time_two)
    scheduler.step()  #动态学习率更新 
#如果你不是非必须，我建议你尽量不要使用n折交叉验证，使用数据增强可能效果更优于它。

epoch: 0
The Model-Train-Time spent  0 min 37.20 s
Accuracy : Train is 0.6972861842105263 , Valid is 0.6908967391304348 ;  Loss : Train is  0.8630717970822986 ,Valid is 0.8829169454781906
覆盖最好的模型...
epoch: 1
The Model-Train-Time spent  0 min 29.02 s
Accuracy : Train is 0.8246710526315789 , Valid is 0.7978940217391304 ;  Loss : Train is  0.5024037125863527 ,Valid is 0.5773420178371927
覆盖最好的模型...
epoch: 2
The Model-Train-Time spent  0 min 29.18 s
Accuracy : Train is 0.8672697368421053 , Valid is 0.826766304347826 ;  Loss : Train is  0.3719214676242126 ,Valid is 0.4952113123043724
覆盖最好的模型...
epoch: 3
The Model-Train-Time spent  0 min 30.97 s
Accuracy : Train is 0.9013157894736842 , Valid is 0.8128396739130435 ;  Loss : Train is  0.2885220061791571 ,Valid is 0.5839965097282244
epoch: 4
The Model-Train-Time spent  0 min 28.92 s
Accuracy : Train is 0.9247532894736842 , Valid is 0.8288043478260869 ;  Loss : Train is  0.21737738016404604 ,Valid is 0.5669296578220699
覆盖最好的模型...
epoch: 5
The Mod

The Model-Train-Time spent  0 min 35.89 s
Accuracy : Train is 0.9999177631578947 , Valid is 0.8603940217391304 ;  Loss : Train is  0.00031246845855524666 ,Valid is 1.3402792992799177
覆盖最好的模型...
epoch: 44
The Model-Train-Time spent  0 min 33.88 s
Accuracy : Train is 1.0 , Valid is 0.858695652173913 ;  Loss : Train is  5.057336468445627e-06 ,Valid is 1.3666388444278552
epoch: 45
The Model-Train-Time spent  0 min 33.82 s
Accuracy : Train is 1.0 , Valid is 0.8607336956521738 ;  Loss : Train is  6.71081637081347e-07 ,Valid is 1.39637967037118
覆盖最好的模型...
epoch: 46
The Model-Train-Time spent  0 min 32.91 s
Accuracy : Train is 1.0 , Valid is 0.8627717391304348 ;  Loss : Train is  1.1465071063292653e-06 ,Valid is 1.3972815923068835
覆盖最好的模型...
epoch: 47
The Model-Train-Time spent  0 min 33.53 s
Accuracy : Train is 1.0 , Valid is 0.8610733695652174 ;  Loss : Train is  3.1018237534322237e-06 ,Valid is 1.4343394326127095
epoch: 48
The Model-Train-Time spent  0 min 37.56 s
Accuracy : Train is 1.0 , 

Accuracy : Train is 1.0 , Valid is 0.8648097826086957 ;  Loss : Train is  3.0390526119031404e-09 ,Valid is 1.4510513233101887
epoch: 90
The Model-Train-Time spent  0 min 29.86 s
Accuracy : Train is 1.0 , Valid is 0.8651494565217391 ;  Loss : Train is  5.627149029781944e-09 ,Valid is 1.445351191188978
epoch: 91
The Model-Train-Time spent  0 min 29.25 s
Accuracy : Train is 1.0 , Valid is 0.8637907608695652 ;  Loss : Train is  3.6272563432392322e-09 ,Valid is 1.4967239810072857
epoch: 92
The Model-Train-Time spent  0 min 29.36 s
Accuracy : Train is 1.0 , Valid is 0.8661684782608695 ;  Loss : Train is  6.3035833208184495e-09 ,Valid is 1.388577015503593
epoch: 93
The Model-Train-Time spent  0 min 28.18 s
Accuracy : Train is 1.0 , Valid is 0.8654891304347826 ;  Loss : Train is  3.127283171603554e-09 ,Valid is 1.4440702666407046
epoch: 94
The Model-Train-Time spent  0 min 28.32 s
Accuracy : Train is 1.0 , Valid is 0.8627717391304348 ;  Loss : Train is  3.6272563432392322e-09 ,Valid is 1.46296

The Model-Train-Time spent  0 min 28.52 s
Accuracy : Train is 1.0 , Valid is 0.8671875 ;  Loss : Train is  5.882037313360917e-10 ,Valid is 1.4224430659542913
epoch: 137
The Model-Train-Time spent  0 min 28.13 s
Accuracy : Train is 1.0 , Valid is 0.8668478260869565 ;  Loss : Train is  5.293833582024825e-10 ,Valid is 1.4486410255017488
epoch: 138
The Model-Train-Time spent  0 min 28.30 s
Accuracy : Train is 1.0 , Valid is 0.8678668478260869 ;  Loss : Train is  1.1469972761053789e-09 ,Valid is 1.4599806666374207
epoch: 139
The Model-Train-Time spent  0 min 29.16 s
Accuracy : Train is 1.0 , Valid is 0.8648097826086957 ;  Loss : Train is  3.5292223880165503e-10 ,Valid is 1.4554084746733955
epoch: 140
The Model-Train-Time spent  0 min 28.82 s
Accuracy : Train is 1.0 , Valid is 0.8637907608695652 ;  Loss : Train is  5.784003358138235e-10 ,Valid is 1.4913692059724226
epoch: 141
The Model-Train-Time spent  0 min 29.17 s
Accuracy : Train is 1.0 , Valid is 0.8634510869565217 ;  Loss : Train is  7

The Model-Train-Time spent  0 min 35.07 s
Accuracy : Train is 1.0 , Valid is 0.8641304347826086 ;  Loss : Train is  3.921358208907278e-11 ,Valid is 1.5011517405509949
epoch: 184
The Model-Train-Time spent  0 min 33.73 s
Accuracy : Train is 1.0 , Valid is 0.8648097826086957 ;  Loss : Train is  8.823055970041376e-11 ,Valid is 1.503134566804637
epoch: 185
The Model-Train-Time spent  0 min 34.08 s
Accuracy : Train is 1.0 , Valid is 0.8661684782608695 ;  Loss : Train is  1.1764074626721834e-10 ,Valid is 1.4820419342621514
epoch: 186
The Model-Train-Time spent  0 min 35.08 s
Accuracy : Train is 1.0 , Valid is 0.8668478260869565 ;  Loss : Train is  1.8626451492309571e-10 ,Valid is 1.460241947485053
epoch: 187
The Model-Train-Time spent  0 min 33.82 s
Accuracy : Train is 1.0 , Valid is 0.8675271739130435 ;  Loss : Train is  2.4508488805670486e-10 ,Valid is 1.4490218188451685
epoch: 188
The Model-Train-Time spent  0 min 32.27 s
Accuracy : Train is 1.0 , Valid is 0.8641304347826086 ;  Loss : Tra